In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

In [2]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

## Loading Tasks and Agents YAML files

In [4]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [5]:
agents_config

{'project_planning_agent': {'role': 'The Ultimate Project Planner\n',
  'goal': 'To meticulously break down the {project_type} project into actionable tasks, ensuring no detail is overlooked, and setting precise timelines that align with the {project_objectives}.\n',
  'backstory': "As a veteran project manager, youâ€™ve led numerous successful projects, particularly in {industry}. Your keen eye for detail and strategic thinking have always ensured that projects are delivered on time and within scope. Now, you're tasked with planning the next groundbreaking {project_type} project.\n",
  'allow_delegation': False,
  'verbose': True},
 'estimation_agent': {'role': 'Expert Estimation Analyst\n',
  'goal': 'Provide highly accurate time, resource, and effort estimations for each task in the {project_type} project to ensure it is delivered efficiently and on budget.\n',
  'backstory': 'You are the go-to expert for project estimation in {industry}. With a wealth of experience and access to va

In [6]:
tasks_config

{'task_breakdown': {'description': "Carefully analyze the project_requirements for the {project_type} project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:\n{project_requirements}\n\nTeam members:\n{team_members}\n",
  'expected_output': 'A comprehensive list of tasks with detailed descriptions, timelines, dependencies, and deliverables. Your final output MUST include a Gantt chart or similar timeline visualization specific to the {project_type} project.\n'},
 'time_resource_estimation': {'description': 'Thoroughly evaluate each task in the {project_type} project to estimate the time, resources, and effort required. Use historical data, task complexity, and available resources to provide a realistic estimation for each task.\n',
  'expected_output': 'A detailed estimation report outlining the time, resources, and effort required for each task in the {project_type} project. You

## Create Pydantic Models for Structured Output

In [8]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

## Create Crew, Agents and Tasks

In [9]:
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent']
)

estimation_agent = Agent(
  config=agents_config['estimation_agent']
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent']
)

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

Checking env var OPENAI_API_KEY: sk-None-ESgZG75bQBYfzmesEfc9T3BlbkFJqFQHm3phgqUzZFPSSwGm
Mapped key name: api_key
Checking env var OPENAI_API_KEY: sk-None-ESgZG75bQBYfzmesEfc9T3BlbkFJqFQHm3phgqUzZFPSSwGm
Mapped key name: api_key
Checking env var OPENAI_API_KEY: sk-None-ESgZG75bQBYfzmesEfc9T3BlbkFJqFQHm3phgqUzZFPSSwGm
Mapped key name: api_key


In [10]:
# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

### Crew Inputs

In [11]:
from IPython.display import display, Markdown

project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

In [12]:
# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



### Kicking off the crew

In [13]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build tru

### Usage metrics and costs

In [14]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0025


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,16978,10773,6205,9


### Result

In [15]:
result.pydantic.dict()

{'tasks': [{'task_name': 'Create a Responsive Design',
   'estimated_time_hours': 40.0,
   'required_resources': ['Bob Smith']},
  {'task_name': 'Implement a Modern User Interface',
   'estimated_time_hours': 80.0,
   'required_resources': ['Jane Doe', 'Bob Smith']},
  {'task_name': 'Develop a User-Friendly Navigation System',
   'estimated_time_hours': 20.0,
   'required_resources': ['Jane Doe']},
  {'task_name': "Create 'About Us' Page",
   'estimated_time_hours': 20.0,
   'required_resources': ['Bob Smith']},
  {'task_name': "Design a 'Services' Page",
   'estimated_time_hours': 20.0,
   'required_resources': ['Bob Smith']},
  {'task_name': "Create 'Contact Us' Page",
   'estimated_time_hours': 20.0,
   'required_resources': ['Jane Doe']},
  {'task_name': 'Implement a Blog Section',
   'estimated_time_hours': 40.0,
   'required_resources': ['Jane Doe']},
  {'task_name': 'Ensure Fast Loading Times and Optimize SEO',
   'estimated_time_hours': 40.0,
   'required_resources': ['Jane Doe

In [16]:
tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,task_name,estimated_time_hours,required_resources
0,Create a Responsive Design,40.000000,['Bob Smith']
1,Implement a Modern User Interface,80.000000,"['Jane Doe', 'Bob Smith']"
2,Develop a User-Friendly Navigation System,20.000000,['Jane Doe']
3,Create 'About Us' Page,20.000000,['Bob Smith']
4,Design a 'Services' Page,20.000000,['Bob Smith']
5,Create 'Contact Us' Page,20.000000,['Jane Doe']
6,Implement a Blog Section,40.000000,['Jane Doe']
7,Ensure Fast Loading Times and Optimize SEO,40.000000,"['Jane Doe', 'Alice Johnson']"
8,Integrate Social Media Links,20.000000,['Jane Doe']
9,Include Testimonials Section,20.000000,"['Jane Doe', 'Bob Smith']"


### Inspecting Milestones

In [17]:
milestones = result.pydantic.dict()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,milestone_name,tasks
0,Design Phase Completion,"['1', '2', '4', '5', '6']"
1,Development Phase Completion,"['3', '7', '9']"
2,Optimization and Launch,"['8', '10']"
